# Image-to-3D Quickstart (Colab)\n이 노트북은 단일 이미지에서 TripoSR 기반 STL 생성을 빠르게 검증합니다.

In [ ]:
!git clone https://github.com/VAST-AI-Research/TripoSR.git\n%cd TripoSR\n!pip install -q -r requirements.txt\n!pip install -q rembg trimesh onnxruntime "huggingface_hub>=0.24.0" "transformers>=4.41.0"

In [ ]:
import torch\nif torch.cuda.is_available():\n    print('GPU:', torch.cuda.get_device_name(0))\nelse:\n    raise RuntimeError('GPU를 찾을 수 없습니다. 런타임 유형에서 GPU를 선택하세요.')

In [ ]:
from google.colab import files\nfrom PIL import Image\nfrom rembg import remove\n\nuploaded = files.upload()\ninput_filename = list(uploaded.keys())[0]\n\noriginal = Image.open(input_filename)\nno_bg = remove(original)\nwhite_bg = Image.new('RGBA', no_bg.size, (255, 255, 255, 255))\nwhite_bg.paste(no_bg, mask=no_bg.split()[3])\nprocessed_image = white_bg.convert('RGB')

In [ ]:
import torch\nimport numpy as np\nimport trimesh\nfrom tsr.system import TSR\n\nmodel = TSR.from_pretrained('stabilityai/TripoSR', config_name='config.yaml', weight_name='model.ckpt')\nmodel.renderer.set_chunk_size(8192)\nmodel.to('cuda:0')\n\nwith torch.no_grad():\n    scene_codes = model([processed_image], device='cuda:0')\n    meshes = model.extract_mesh(scene_codes, resolution=256, has_vertex_color=False)\n\nraw = meshes[0]\ndef to_numpy(v):\n    if hasattr(v, 'detach'): v = v.detach()\n    if hasattr(v, 'cpu'): v = v.cpu()\n    if hasattr(v, 'numpy'): return v.numpy()\n    return np.asarray(v)\n\nmesh = trimesh.Trimesh(vertices=to_numpy(raw.vertices), faces=to_numpy(raw.faces))\ntrimesh.repair.fix_normals(mesh)\ntrimesh.repair.fix_winding(mesh)\ntrimesh.repair.fill_holes(mesh)\nmesh.merge_vertices()\n\ntarget_height = 100.0\nmesh.apply_scale(target_height / mesh.extents.max())\nmesh.apply_translation([0.0, 0.0, -mesh.bounds[0][2]])\n\nout = 'figure_printable.stl'\nmesh.export(out)\nprint('saved:', out, 'watertight:', mesh.is_watertight)